In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

AIBOX = LOCAL_RTS_DATA + 'aibox-vectors'

OUTDIR = 'data'

/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import emv
import emv.utils
import emv.io.media
import emv.features.audio
import emv.features.text

LOG = emv.utils.get_logger()

In [5]:
# df = rts.metadata.load_metadata_hdf5(METADATA)
# aidf = rts.metadata.link_aivectors_to_metadata(df, AIBOX)
# rts.utils.dataframe_to_hdf5(LOCAL_RTS_DATA + '/metadata', 'rts_aivectors', aidf)

df = emv.utils.dataframe_from_hdf5(LOCAL_RTS_DATA + '/metadata', 'rts_aivectors')
df.head()

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,...,rights,title,resume,geoTheme,resumeSequence,created,published,sequences,aivector_id,aivector_path
mediaId,,,,,,,,,,,,,,,,,,,,,
ZE004015,AA1104002390,/mnt/rts/5/1/0/ZE004015,4437,16:9,SD,2011-04-09T00:00:00Z,Programme,Programme,"Pop et rock, clips",None,...,Restriction/Condition,WALDER / PAMPLEMOUSSE / NICOLAS FRAISSINET - 1...,"* Lausanne : 20110409, magazine de la scène mu...",[LAUSANNE],"[00:01.22\nLausanne, MAD, diverses séquences :...",1670505072,1302307200,"{'901697': {'guid': 'AA1104002390', 'mediaId':...",0b3241369e6e4a20942adc020470326f,f/6/2/0b3241369e6e4a20942adc020470326f
ZT001013,AA1110016880,/mnt/rts/3/1/0/ZT001013,1675,16:9,SD,2011-10-31T00:00:00Z,Programme,Programme,Autres émissions d'actualités,None,...,Inconnu,STUPEUR A NYON // DE LA SUEUR ET DES THUNES //...,"* Vaud : 20111031 et archives, les employés du...",None,[20:15.41\r\nPlateau : diverses présentations ...,1670505072,1320019200,"{'899747': {'guid': 'AA1110016880', 'mediaId':...",67e6386cff0e4b6f860f29b8aea063c1,1/c/3/67e6386cff0e4b6f860f29b8aea063c1
ZE003020,AA1110017160,/mnt/rts/0/2/0/ZE003020,1465,16:9,SD,2010-11-02T00:00:00Z,Programme,Programme,"Jeux, concours",None,...,Inconnu,10.11.02,None,None,None,1670505072,1288656000,{},6a6ec4662d42413a8f2bf383929c8580,0/8/5/6a6ec4662d42413a8f2bf383929c8580
ZM002012,AA1203025010,/mnt/rts/2/1/0/ZM002012,36,16:9,HD,2012-03-22T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,...,Inconnu,Magazine culturel itinérant. BL PUCE 2203,None,None,None,1670505073,1332374400,{},acb248bd411749cf97758783fab376a2,2/a/6/acb248bd411749cf97758783fab376a2
ZM003016,AA1206030870,/mnt/rts/6/1/0/ZM003016,60,16:9,HD,2012-06-25T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,...,Inconnu,BLS LUNDI 2506,None,None,None,1670505073,1340582400,{},7275a30f0fd64af4998840e8c3bd7c67,7/6/c/7275a30f0fd64af4998840e8c3bd7c67
